In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                      homepage     id  \
0  http://www.avatarmovie.com/  19995   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   

  original_title                                           overview  \
0         Avatar  In the 22nd century, a paraplegic Marine is di...   

   popularity                               production_companies  \
0  150.437577  [{"name": "Ingenious Film Partners", "id": 289...   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   

                       tagline   title  vote_average  vote_count  
0  Enter the World of Pandora.  Avatar           7.2       11800

In [4]:
movies=movies.merge(credits,on='title')

In [5]:
movies.head(1)

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                      homepage     id  \
0  http://www.avatarmovie.com/  19995   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   

  original_title                                           overview  \
0         Avatar  In the 22nd century, a paraplegic Marine is di...   

   popularity                               production_companies  ... runtime  \
0  150.437577  [{"name": "Ingenious Film Partners", "id": 289...  ...   162.0   

                                    spoken_languages    status  \
0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   

                       tagline   title vote_average vote_count movie_id  \
0  Enter the World of Pandora.  Avatar          7.2      11800    19995   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  

[1 rows x 23 columns]

In [6]:
# we aregoing to keep geners,id,keywords,title,overview,casts,crews
#original lan ==not our wrk because there we not get any information which he;ps us to creating our
#tag because ther are 100-movie in different lang from that we cannot get any conclusion 
#matlab jada maxx movie eng me hi hai
movies['original_language'].value_counts()

en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [7]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [8]:
#start preprocessing
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
#remove missing data
movies.dropna(inplace=True)

In [10]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
#check for duplicate
movies.duplicated().sum()

0

In [12]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"},
#{"id": 878, "name": "Science Fiction"}]'
#Look above our genres is in very weired form so only take data of genres name which is in our work

# so our task is to convert these genrs in that way   ['action,adventer,fantasy,scienc  fic']

#Also see guys this genere is in string of list so we use a built in function (LITERAL_EVL which is in AST module)
#which convert string of lis into a list

#so amke a helper function for doing this task
import ast
def convert(obj):
    Lis=[]
    for i in ast.literal_eval(obj):
        Lis.append(i['name'])
    return Lis

In [14]:
movies['genres']=movies['genres'].apply(convert)

In [15]:
movies['keywords']=movies['keywords'].apply(convert)

In [16]:
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [17]:
#now task is to get top three cast because it is helpful for making a 
#reccommendation system because they gives more accouracy because people only remember tops
#cast in a film not remember a people who are doing a side role

#and also one thing the in cast dic the name is only
#for our work so again with the help of helper func we get that particular info

def helperfun(obj):
    lis=[]
    counter=0
    for i in ast.literal_eval(obj):
        if(counter!=3):
            lis.append(i['name'])
            counter+=1
        else:
            break
    return lis

In [18]:
movies['cast']=movies['cast'].apply(helperfun)

In [19]:
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast  \
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]   
1     [Johnny Depp, Orlando Bloom, Keira Knightley]   
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]   
3      [Christian Bale, Michael Caine, Gary Oldman]   
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

In [20]:
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [21]:
#now we need name of job role belong to director

def fetch_director_name(obj):
    lis=[]
    for i in ast.literal_eval(obj):
        if (i['job']=='Director'):
            lis.append(i['name'])
            break
    return lis

In [22]:
movies['crew']=movies['crew'].apply(fetch_director_name)

In [23]:
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast                 crew  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]     [Gore Verbinski]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]         [Sam Mendes]  
3      [Christian Bale, Michael Caine, Gary Oldman]  [Christopher Nolan]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]     [Andrew Stanton]

In [24]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [25]:
#convert this it into a list
#here we use lambda funtion and i think you guys famlier with lambda 1 line func.....

movies['overview']=movies['overview'].apply(lambda x:x.split())

In [26]:
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast                 crew  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]     [Gore Verbinski]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]         [Sam Mendes]  
3      [Christian Bale, Michael Caine, Gary Oldman]  [Christopher Nolan]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]     [Andrew Stanton]

In [27]:
#Sam Worthington===now there is proble if we create a tag #sam one and other #worthington but 
#if name=>sam wor and sam bhai then our recommendation system can give wrong ans so remove the space
#between sam wor and sam bhai

#also do this for geners,keyword,cast,crew


movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [28]:
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

                                            keywords  \
0  [cultureclash, future, spacewar, spacecolony, ...   
1  [ocean, drugabuse, exoticisland, eastindiatrad...   
2  [spy, basedonnovel, secretagent, sequel, mi6, ...   
3  [dccomics, crimefighter, terrorist, secretiden...   
4  [basedonnovel, mars, medallion, spacetravel, p...   

                                            cast                crew  
0  [SamWorthington, ZoeSaldana, SigourneyWeaver]      [JamesCameron]  
1     [JohnnyDepp, OrlandoBloom, KeiraKnightley]     [GoreVerbinski]  
2      [DanielCraig, ChristophWaltz, LéaSeydoux]         [SamMendes]  
3      [ChristianBale, MichaelCaine, GaryOldman]  [ChristopherNolan]  
4    [TaylorKitsch, LynnCollins, SamanthaMorton]     [AndrewStanton]

In [29]:
#now our tast is to creating a tag so we make a new column of tag of
#tag==overview+cast+crew+genres+keyword

movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [30]:
##create a new data set of title,movies id,tag

new_dataset=movies[['movie_id','title','tags']]

In [31]:
new_dataset.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  [In, the, 22nd, century,, a, paraplegic, Marin...  
1  [Captain, Barbossa,, long, believed, to, be, d...  
2  [A, cryptic, message, from, Bond’s, past, send...  
3  [Following, the, death, of, District, Attorney...  
4  [John, Carter, is, a, war-weary,, former, mili...

In [32]:
#convert tag which is in list convert it into string
  
new_dataset['tags']=new_dataset['tags'].apply(lambda x:" ".join(x))

C:\Users\maury\AppData\Local\Temp\ipykernel_14040\2589936506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].apply(lambda x:" ".join(x))


In [33]:
new_dataset.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...

In [34]:
new_dataset['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [35]:
#convert it into lowercase

new_dataset['tags']=new_dataset['tags'].apply(lambda x:x.lower())


C:\Users\maury\AppData\Local\Temp\ipykernel_14040\458408578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].apply(lambda x:x.lower())


In [36]:
new_dataset.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  in the 22nd century, a paraplegic marine is di...  
1  captain barbossa, long believed to be dead, ha...  
2  a cryptic message from bond’s past sends him o...  
3  following the death of district attorney harve...  
4  john carter is a war-weary, former military ca...

In [37]:
#WE NEED TO DO A TEXT VECTORIZATION=we do it by bags of word technique
#WHY WE NEED this lets understand our first prob we have to create a mode who reccomend similar movies on the basics of tags
#thats means we are are comparing two tags and get the score of similiarity of two tags on the basis of thiresimilarity in words
#but on the basics no of similar word it is not a good ideas so we do text vectorization 

#note we dont need stop words so remove it  and use sklearn and countvectorizatiom class

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [38]:
#convert this into a numpy arr
vectors=cv.fit_transform(new_dataset['tags']).toarray()

In [39]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [40]:
#maximum no zero hai q ki 1 movie me 5000 words imposible hai so
#lets see words include in 5000 words
cv.get_feature_names()

C:\Users\maury\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1930s',
 '1940s',
 '1950',
 '1950s',
 '1960s',
 '1970s',
 '1980',
 '1980s',
 '1985',
 '1990s',
 '1999',
 '19th',
 '19thcentury',
 '20',
 '200',
 '2009',
 '20th',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '60s',
 '70',
 '70s',
 'aaron',
 'aaroneckhart',
 'abandoned',
 'abducted',
 'abigailbreslin',
 'abilities',
 'ability',
 'able',
 'aboard',
 'abuse',
 'abusive',
 'academy',
 'accept',
 'accepted',
 'accepts',
 'access',
 'accident',
 'accidental',
 'accidentally',
 'accompanied',
 'accomplish',
 'account',
 'accountant',
 'accused',
 'ace',
 'achieve',
 'act',
 'acting',
 'action',
 'actionhero',
 'actions',
 'activist',
 'activities',
 'activity',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adamsandler',
 'adamshankman',
 'adaptation',
 'adapted',
 'addict',
 'addicted',
 'addiction',
 'adolescence',
 'adolescent'

In [41]:
#amid',
#'amidst'===see the both meaning are same so remove these repeting word by using stamin technique
#basically stamin convert similar word into one word like dancing ,danced,dance==danc
#we will do this by using nltp libraries
import nltk

In [42]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [43]:
def stem(text):
    lis=[]
    for i in text.split():
        lis.append(ps.stem(i))
    return " ".join(lis)

In [44]:
new_dataset['tags']=new_dataset['tags'].apply(stem)

C:\Users\maury\AppData\Local\Temp\ipykernel_14040\1283114390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].apply(stem)


In [45]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [46]:
cv.get_feature_names()

['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1930s',
 '1940s',
 '1950',
 '1950s',
 '1960s',
 '1970s',
 '1980',
 '1980s',
 '1985',
 '1990s',
 '1999',
 '19th',
 '19thcentury',
 '20',
 '200',
 '2009',
 '20th',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '60s',
 '70',
 '70s',
 'aaron',
 'aaroneckhart',
 'abandoned',
 'abducted',
 'abigailbreslin',
 'abilities',
 'ability',
 'able',
 'aboard',
 'abuse',
 'abusive',
 'academy',
 'accept',
 'accepted',
 'accepts',
 'access',
 'accident',
 'accidental',
 'accidentally',
 'accompanied',
 'accomplish',
 'account',
 'accountant',
 'accused',
 'ace',
 'achieve',
 'act',
 'acting',
 'action',
 'actionhero',
 'actions',
 'activist',
 'activities',
 'activity',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adamsandler',
 'adamshankman',
 'adaptation',
 'adapted',
 'addict',
 'addicted',
 'addiction',
 'adolescence',
 'adolescent'

In [47]:
#here we are using cosin similarity for calculationg distance not eucledian distance 
#cause for lage dimenssion accuracy of eucledian is poor
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
similarity=cosine_similarity(vectors)

In [49]:
similarity[1]

array([0.08964215, 1.        , 0.0625    , ..., 0.02635231, 0.        ,
       0.        ])

In [50]:
#this similarity is the disc b/w own vs other 5000 movies so shape will be (5000*5000)
#lets find the top 5 match movie

#our ans will be top maximum 5 value in similarity[1] so if sorted it we get easily top five value but index wiil be change
#so we used here enumerator and we use sorted and put key=2nd wala
#sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

def recommend(movie):
    movie_index=new_dataset[new_dataset['title']==movie].index[0]
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_dataset.iloc[i[0]].title)
    

In [51]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [53]:
#now dump the model
import pickle

In [55]:
pickle.dump(new_dataset.to_dict(),open('movies_dict.pkl','wb'))

In [57]:
#also dum similiraity which need in reccomending
pickle.dump(similarity,open('similarity.pkl','wb'))

In [58]:
pwd

'C:\\Users\\maury'